## Building features for user level expense anomaly detection

In [1]:
import pandas as pd 
df = pd.read_csv("../../data/raw/expenses.csv")
df.head(10)

,expenseId,groupId,paidBy,amount,category,createdAt
0,E1,G3,U5,14737,rent,2025-03-24 08:42:00
1,E2,G2,U5,292,food,2025-03-09 13:28:00
2,E3,G2,U4,7349,rent,2025-01-05 11:22:00
3,E4,G2,U5,513,food,2025-03-12 08:02:00
4,E5,G2,U2,1052,other,2025-05-14 05:33:00
5,E6,G3,U2,3647,other,2025-01-18 05:13:00
6,E7,G2,U2,2376,travel,2025-01-09 21:11:00
7,E8,G2,U2,1795,travel,2025-02-06 07:42:00
8,E9,G1,U5,517,travel,2025-05-07 18:01:00
9,E10,G1,U1,7345,rent,2025-05-11 09:37:00


In [2]:
df["createdAt"] = pd.to_datetime(df["createdAt"])
df = df.sort_values("createdAt")
df

,expenseId,groupId,paidBy,amount,category,createdAt
201,E202,G2,U5,2258,travel,2025-01-01 06:58:00
291,E292,G1,U3,4884,other,2025-01-01 12:35:00
191,E192,G3,U2,5761,shopping,2025-01-01 17:32:00
280,E281,G2,U4,2080,other,2025-01-01 21:40:00
350,E351,G3,U5,499,food,2025-01-02 10:11:00
...,...,...,...,...,...,...
93,E94,G2,U1,589,food,2025-06-29 08:33:00
60,E61,G2,U2,2645,other,2025-06-29 14:22:00
384,E385,G3,U1,2048,other,2025-06-30 07:45:00
143,E144,G1,U5,2732,travel,2025-06-30 11:41:00


In [3]:
user_stats = {} # Store running stats per user
past_counts  = [] # Store computed feature for each row
user_avg_amounts = [] # Total / count
amount_minus_user_avg = [] # current - past avg
time_gaps = []

In [4]:
# Based only on past behavior, what is normal for this user?
for _, row in df.iterrows():
    user = row['paidBy']
    amount = row["amount"]
    current_timestamp = row['createdAt']
    
    # if user is seen before
    if user in user_stats:
        past_count = user_stats[user]['count']
        past_sum = user_stats[user]['sum']
        last_timestamp = user_stats[user]['last_timestamp']

        user_avg = past_sum / past_count
        time_gap = (current_timestamp - last_timestamp).total_seconds() / 60
    else: 
        past_count = 0
        user_avg = 0
        time_gap = 0

    # Storing the features
    past_counts.append(past_count)
    user_avg_amounts.append(user_avg)
    amount_minus_user_avg.append(amount - user_avg)
    time_gaps.append(time_gap)

    # update dict after computing features
    if user in user_stats:
        user_stats[user]['count'] += 1
        user_stats[user]['sum'] += amount
        user_stats[user]['last_timestamp'] = current_timestamp
    else:
         user_stats[user] = {
            "count": 1,
            "sum": amount,
            'last_timestamp' : current_timestamp
        }

df["past_transaction_count"] = past_counts
df["user_avg_amount"] = user_avg_amounts
df["amount_minus_user_avg"] = amount_minus_user_avg
df['time_gap_minutes'] = time_gaps
df.head(20)

,expenseId,groupId,paidBy,amount,category,createdAt,past_transaction_count,user_avg_amount,amount_minus_user_avg,time_gap_minutes
201,E202,G2,U5,2258,travel,2025-01-01 06:58:00,0,0.000000,2258.000000,0.0
291,E292,G1,U3,4884,other,2025-01-01 12:35:00,0,0.000000,4884.000000,0.0
191,E192,G3,U2,5761,shopping,2025-01-01 17:32:00,0,0.000000,5761.000000,0.0
280,E281,G2,U4,2080,other,2025-01-01 21:40:00,0,0.000000,2080.000000,0.0
350,E351,G3,U5,499,food,2025-01-02 10:11:00,1,2258.000000,-1759.000000,1633.0
119,E120,G3,U4,7388,shopping,2025-01-02 13:45:00,1,2080.000000,5308.000000,965.0
260,E261,G2,U5,10212,travel,2025-01-03 02:45:00,2,1378.500000,8833.500000,994.0
450,E451,G2,U4,5907,rent,2025-01-03 10:41:00,2,4734.000000,1173.000000,1256.0
315,E316,G3,U2,8668,rent,2025-01-03 11:57:00,1,5761.000000,2907.000000,2545.0
334,E335,G3,U3,772,food,2025-01-03 12:12:00,1,4884.000000,-4112.000000,2857.0


In [5]:
df['hour'] = pd.to_datetime(df['createdAt']).dt.hour
df['date'] = pd.to_datetime(df['createdAt']).dt.date

In [6]:
df['date'] = pd.to_datetime(df['date']).dt.floor('D')
df['day_of_week'] = df['date'].dt.dayofweek


In [7]:
df = df.drop(columns=['expenseId','groupId', 'paidBy', 'date', 'createdAt', 'category'])

In [10]:
df.isna().sum()

amount                    0
past_transaction_count    0
user_avg_amount           0
amount_minus_user_avg     0
time_gap_minutes          0
hour                      0
day_of_week               0
dtype: int64

In [11]:
df.to_csv('../../data/processed/features.csv')